In [1]:
import os
import pandas as pd
from transliterate import translit, get_available_language_codes
import pyarrow as pa
from pyarrow import parquet as pq

In [2]:
train_file_path=os.getcwd()+"\\Data\\sales_train.pq"
test_file_path=os.getcwd()+"\\Data\\test.csv"
sub_file_path=os.getcwd()+"\\Data\\sample_submission.csv"
shops_file_path=os.getcwd()+"\\Data\\shops.csv"
items_file_path=os.getcwd()+"\\Data\\items.csv"
item_cats_file_path=os.getcwd()+"\\Data\\item_categories.csv"

### Read in the data and transliterate (RU -> EN) where required

In [3]:
train_table=pq.read_table(train_file_path)
df_train=train_table.to_pandas()
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
df_test=pd.read_csv(test_file_path)
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
df_sub=pd.read_csv(sub_file_path)
df_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [6]:
#get_available_language_codes()

In [7]:
df_shops=pd.read_csv(shops_file_path)
df_shops['shop_name']=[translit(name, 'ru', reversed=True) for name in df_shops.shop_name]
df_shops.head()

,shop_name,shop_id
0,"!Jakutsk Ordzhonikidze, 56 fran",0
1,"!Jakutsk TTs ""Tsentral'nyj"" fran",1
2,"Adygeja TTs ""Mega""",2
3,"Balashiha TRK ""Oktjabr'-Kinomir""",3
4,"Volzhskij TTs ""Volga Moll""",4


In [8]:
df_items=pd.read_csv(items_file_path)
df_items['item_name']=[translit(name, 'ru', reversed=True) for name in df_items.item_name]
df_items.head()

,item_name,item_id,item_category_id
0,! VO VLASTI NAVAZhDENIJa (PLAST.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***V LUChAH SLAVY (UNV) D,2,40
3,***GOLUBAJa VOLNA (Univ) D,3,40
4,***KOROBKA (STEKLO) D,4,40


In [9]:
df_item_cats=pd.read_csv(item_cats_file_path)
df_item_cats['item_category_name']=[translit(name, 'ru', reversed=True) for name in df_item_cats.item_category_name]
df_item_cats.head()

,item_category_name,item_category_id
0,PC - Garnitury/Naushniki,0
1,Aksessuary - PS2,1
2,Aksessuary - PS3,2
3,Aksessuary - PS4,3
4,Aksessuary - PSP,4


### Join the files to get the expanded training set

In [10]:
df_train_enhanced=pd.merge(df_train, df_shops, on='shop_id', how='left')
df_train_enhanced=pd.merge(df_train_enhanced, df_items, on='item_id', how='left')
df_train_enhanced=pd.merge(df_train_enhanced, df_item_cats, on='item_category_id', how='left')

### Join to [shop-id] tuples

In [11]:
df_test=pd.read_csv(test_file_path)
print(len(df_test))
df_test.head()

214200


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [12]:
df_train_enhanced=pd.merge(df_train_enhanced, df_test, on=['shop_id', 'item_id'], how='right').drop(['shop_id', 'item_id'], axis=1)
df_train_enhanced.head()

,date,date_block_num,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name,ID
0,21.09.2014,20.0,2599.0,1.0,"Vologda TRTs ""Marmelad""","NHL 15 [PS3, russkie subtitry]",19.0,Igry - PS3,0
1,29.11.2014,22.0,2599.0,1.0,"Vologda TRTs ""Marmelad""","NHL 15 [PS3, russkie subtitry]",19.0,Igry - PS3,0
2,28.12.2014,23.0,1999.0,1.0,"Vologda TRTs ""Marmelad""","NHL 15 [PS3, russkie subtitry]",19.0,Igry - PS3,0
3,20.12.2014,23.0,1999.0,1.0,"Vologda TRTs ""Marmelad""","NHL 15 [PS3, russkie subtitry]",19.0,Igry - PS3,0
4,02.01.2015,24.0,1999.0,1.0,"Vologda TRTs ""Marmelad""","NHL 15 [PS3, russkie subtitry]",19.0,Igry - PS3,0


### Sort by ascending date

In [13]:
df_train_enhanced['date']=pd.to_datetime(df_train_enhanced.date, format='%d.%m.%Y')
df_train_enhanced.sort_values('date', inplace=True)

### Deal with missing values for IDs - zeroise item_cnt_day

In [14]:
df_train_enhanced[df_train_enhanced.date.isnull()].sort_values('date', ascending=False).head()

,date,date_block_num,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name,ID
13,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
24,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
96,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
342,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16
414,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21


In [15]:
df_train_enhanced.item_cnt_day=df_train_enhanced.item_cnt_day.fillna(0)

### Write out the training set for EDA

In [16]:
out_path=os.getcwd()+"\\Training\\training_set.pq"
table_train_enhanced=pa.Table.from_pandas(df_train_enhanced.set_index('date'))
pq.write_table(table_train_enhanced, out_path)

In [17]:
len(df_train_enhanced)

1327235